In [2]:
from pysr import PySRRegressor
import kan
import torch
import numpy as np
from libraries import tfim_functions
from libraries import utils
from libraries import magnetization
import matplotlib.pyplot as plt
import sympy
torch.set_default_dtype(torch.float64)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [3]:
N = 5

In [4]:
input = utils.generate_input_torch(N)
input

tensor([[0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [1., 0., 1., 0., 0.],
        [0., 1., 1., 0., 0.],
        [1., 1., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 1., 0.],
        [0., 1., 0., 1., 0.],
        [1., 1., 0., 1., 0.],
        [0., 0., 1., 1., 0.],
        [1., 0., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [1., 1., 1., 1., 0.],
        [0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 1.],
        [0., 1., 0., 0., 1.],
        [1., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1.],
        [1., 0., 1., 0., 1.],
        [0., 1., 1., 0., 1.],
        [1., 1., 1., 0., 1.],
        [0., 0., 0., 1., 1.],
        [1., 0., 0., 1., 1.],
        [0., 1., 0., 1., 1.],
        [1., 1., 0., 1., 1.],
        [0., 0., 1., 1., 1.],
        [1., 0., 1., 1., 1.],
        [0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]], dtype=torch.float32)

In [5]:
labels = torch.tensor([1 - 2 * (magnetization.count_magnetization(i) % 2) for i in range(2 ** N)])
labels

tensor([ 1, -1, -1,  1, -1,  1,  1, -1, -1,  1,  1, -1,  1, -1, -1,  1, -1,  1,
         1, -1,  1, -1, -1,  1,  1, -1, -1,  1, -1,  1,  1, -1])

In [14]:
model = PySRRegressor(
    maxsize=20,
    niterations=40,  # < Increase me for better results
    binary_operators=["+", "*"],
    unary_operators=[
        "sin",
        # ^ Custom operator (julia syntax)
    ],
    extra_sympy_mappings={"inv": lambda x: 1 / x},
    # ^ Define operator for SymPy as well
    elementwise_loss="loss(prediction, target) = (prediction - target)^2",
    # ^ Custom loss function (julia syntax)
)

In [15]:
model.fit(input, labels)

c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\pysr\sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
[ Info: Started!


───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           1.000e+00  0.000e+00  y = -1.4348e-05
9           1.000e+00  1.490e-08  y = ((x₂ + x₁) * 4.1236e-06) * (x₀ + x₃)
11          9.688e-01  1.587e-02  y = (((x₂ * (x₀ * -1)) * x₁) * x₃) * x₄
12          5.586e-01  5.506e-01  y = sin(((x₀ + x₁) + (x₄ + x₃)) * (x₂ * 15.084))
13          4.694e-01  1.739e-01  y = sin(sin(((x₄ + (x₃ + x₀)) * (x₁ + x₂)) * 20.944))
14          2.130e-10  2.151e+01  y = sin((((x₁ + ((x₄ + x₀) + x₂)) + -0.50275) + x₃) * -3.1...
                                      446)
16          3.119e-12  2.112e+00  y = sin(((((x₀ + x₄) + x₃) + -0.49917) + x₁) * ((x₂ * 6.28...
                                      08) + -3.1402))
18          3.886e-15  3.344e+00  y = sin(((x₂ * 1.9921) + -2.9881) * ((x₀ + (((x₄ + x₃) + -...
                                      0.50006) + x₁)) * -3.1542))
20          1.443e-15  4.952e-01  y 

[ Info: Final population:
[ Info: Results saved to:


PySRRegressor.equations_ = [
	   pick         score                                           equation  \
	0        0.000000e+00                                      -1.4347705e-5   
	1        2.250000e-08             ((x2 + x1) * 4.1235853e-6) * (x0 + x3)   
	2        1.587423e-02        (((x2 * (x0 * -1.0000337)) * x1) * x3) * x4   
	3        5.506419e-01    sin(((x0 + x1) + (x4 + x3)) * (x2 * 15.083596))   
	4        1.738530e-01  sin(sin(((x4 + (x3 + x0)) * (x1 + x2)) * 20.94...   
	5        2.151341e+01  sin((((x1 + ((x4 + x0) + x2)) + -0.50275093) +...   
	6        2.111990e+00  sin(((((x0 + x4) + x3) + -0.49916875) + x1) * ...   
	7        3.343910e+00  sin(((x2 * 1.992136) + -2.9881275) * ((x0 + ((...   
	8  >>>>  4.951994e-01  sin((((x2 * 1.9921299) + -2.9881194) * ((x0 + ...   
	
	           loss  complexity  
	0  9.999999e-01           1  
	1  9.999998e-01           9  
	2  9.687500e-01          11  
	3  5.585615e-01          12  
	4  4.694265e-01          13  
	5  2.130163e-10          14  
	6  3.118616e-12          16  
	7  3.885781e-15          18  
	8  1.443290e-15          20  
]

  - outputs\20250619_143607_T3e6WT\hall_of_fame.csv


In [18]:
model.equations_

,complexity,loss,equation,score,sympy_format,lambda_format
0,1,9.999999e-01,-1.4347705e-5,0.000000e+00,-1.43477050000000e-5,PySRFunction(X=>-0.0000143477050000000)
1,9,9.999998e-01,((x2 + x1) * 4.1235853e-6) * (x0 + x3),2.250000e-08,(x1 + x2)*4.1235853e-6*(x0 + x3),PySRFunction(X=>(x1 + x2)*4.1235853e-6*(x0 + x3))
2,11,9.687500e-01,(((x2 * (x0 * -1.0000337)) * x1) * x3) * x4,1.587423e-02,x2*x0*(-1.0000337)*x1*x3*x4,PySRFunction(X=>x2*x0*(-1.0000337)*x1*x3*x4)
3,12,5.585615e-01,sin(((x0 + x1) + (x4 + x3)) * (x2 * 15.083596)),5.506419e-01,sin((x0 + x1 + x3 + x4)*x2*15.083596),PySRFunction(X=>sin((x0 + x1 + x3 + x4)*x2*15....
4,13,4.694265e-01,sin(sin(((x4 + (x3 + x0)) * (x1 + x2)) * 20.94...,1.738530e-01,sin(sin((x0 + x3 + x4)*(x1 + x2)*20.94397)),PySRFunction(X=>sin(sin((x0 + x3 + x4)*(x1 + x...
5,14,2.130163e-10,sin((((x1 + ((x4 + x0) + x2)) + -0.50275093) +...,2.151341e+01,sin((x0 + x1 + x2 + x3 + x4 - 0.50275093)*(-3....,PySRFunction(X=>sin((x0 + x1 + x2 + x3 + x4 - ...
6,16,3.118616e-12,sin(((((x0 + x4) + x3) + -0.49916875) + x1) * ...,2.111990e+00,sin((x2*6.2808447 - 3.1401684)*(x0 + x1 + x3 +...,PySRFunction(X=>sin((x2*6.2808447 - 3.1401684)...
7,18,3.885781e-15,sin(((x2 * 1.992136) + -2.9881275) * ((x0 + ((...,3.343910e+00,sin((x2*1.992136 - 2.9881275)*(x0 + x1 + x3 + ...,PySRFunction(X=>sin((x2*1.992136 - 2.9881275)*...
8,20,1.443290e-15,sin((((x2 * 1.9921299) + -2.9881194) * ((x0 + ...,4.951994e-01,sin((x2*1.9921299 - 2.9881194)*(x0 + x1 + x3 +...,PySRFunction(X=>sin((x2*1.9921299 - 2.9881194)...


In [ ]:
model.equations_['sympy_format'][8]

sin((x2*1.9921299 - 2.9881194)*(x0 + x1 + x3 + x4 - 0.50003433)*(-3.154157)) - 1.2445909e-8

: 